In [1]:
# pip install beautifulsoup4
# !pip install geopy
# !pip install gpxpy
# !pip install mplleaflet
# !pip install neo4j
# !pip install python-igraph

In [2]:
import gpxpy
import pandas as pd
import geopy
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import mplleaflet
import numpy as np
import glob
import datetime
from datetime import datetime
from datetime import timedelta
import matplotlib.colors as mcolors

In [3]:
gpx_arr=[]
segments=[]
dado = []
for name in glob.glob('gpx/*.gpx'):
    print(name)
    gpx_arr.append(gpxpy.parse(open(name)))
    
for gpx0 in gpx_arr:
    segments.append(gpx0.tracks[0].segments[0])
    
for seg in segments:
    for point_idx, point in enumerate(seg.points):
        dado.append([point.longitude, point.latitude,
                 point.elevation, point.time, seg.get_speed(point_idx)]) 

gpx/20190407.gpx
gpx/20191008.gpx
gpx/20191020.gpx
gpx/20190907.gpx
gpx/20191022.gpx
gpx/20191006.gpx
gpx/20190112.gpx
gpx/20190903.gpx
gpx/20190428.gpx
gpx/20190113.gpx
gpx/20191018.gpx
gpx/20190823.gpx
gpx/20190902.gpx
gpx/20191009.gpx
gpx/20191017.gpx
gpx/20190429.gpx
gpx/20190502.gpx
gpx/20191016.gpx
gpx/20191025.gpx
gpx/20190526.gpx
gpx/20191021.gpx
gpx/20191007.gpx
gpx/20190906.gpx
gpx/20190304.gpx
gpx/20190306.gpx
gpx/20190904.gpx
gpx/20190905.gpx
gpx/20190430.gpx
gpx/20190501.gpx
gpx/20190408.gpx
gpx/20190111.gpx
gpx/20191019.gpx
gpx/20191004.gpx
gpx/20191005.gpx


In [4]:
columns = ['Longitude', 'Latitude', 'Altitude', 'Time', 'Speed']
df = pd.DataFrame(dado, columns=columns)
df.sort_values(by=['Time'], inplace=True)
df['dia_da_semana'] = df.Time.dt.day_name(locale='pt_BR.utf8')

In [5]:
def monta(df):
    lo_origem = []
    la_origem = []
    lo_destino = []
    la_destino = []
    velocidade_media = []
    distancia = []
    altitude_origem = []
    altitude_destino = []
    hora_origem = []
    hora_destino = []
    duracao = []
    data = []
    dia_da_semana = []
    for idx in range(1, len(df)):
        lo_origem.append(df.iloc[idx-1].Longitude)
        la_origem.append(df.iloc[idx-1].Latitude)
        lo_destino.append(df.iloc[idx].Longitude)
        la_destino.append(df.iloc[idx].Latitude)
        distancia.append(geodesic([df.iloc[idx-1].Longitude,df.iloc[idx-1].Latitude],[df.iloc[idx].Longitude, df.iloc[idx].Latitude]).km)
        velocidade_media.append(np.mean([df.iloc[idx-1].Speed,df.iloc[idx].Speed]))
        altitude_origem.append(df.iloc[idx-1].Altitude)
        altitude_destino.append(df.iloc[idx].Altitude)
        data.append(df.iloc[idx]['data'])
        dia_da_semana.append(df.iloc[idx].dia_da_semana)
        hora_origem.append(df.iloc[idx-1].hora)
        hora_destino.append(df.iloc[idx].hora)
        origem = df.iloc[idx-1].hora
        destino = df.iloc[idx].hora
        td = timedelta(hours=destino.hour, minutes=destino.minute, seconds=destino.second)
        to = timedelta(hours=origem.hour, minutes=origem.minute, seconds=origem.second)
        duracao.append(td-to)
    df_trk = pd.DataFrame()
    df_trk['lo_origem'] = lo_origem
    df_trk['la_origem'] = la_origem

    df_trk['lo_destino'] = lo_destino
    df_trk['la_destino'] = la_destino

    df_trk['velocidade_media'] = velocidade_media
    df_trk['distancia'] = distancia

    df_trk['altitude_origem'] = altitude_origem
    df_trk['altitude_destino'] = altitude_destino
    df_trk['data'] = data
    df_trk['dia_da_semana'] = dia_da_semana
    df_trk['hora_origem'] = hora_origem
    df_trk['hora_destino'] = hora_destino
    df_trk['duracao'] = duracao
    
#     df_trk = df_trk.fillna(0)
    return df_trk

In [6]:
df['data']=df.Time.dt.date
df['hora']=df.Time.dt.time
df.drop(columns=['Time'],inplace=True)
dias = list(df.data.unique())
trilhas = []
for dia in dias:
    if(len(df.loc[df.data == dia])>=10):
        trilhas.append(monta(df.loc[df.data == dia]))    

In [7]:
df_trk = pd.DataFrame()
for d in trilhas:
    df_trk = df_trk.append(d, ignore_index = True) 

In [9]:
geolocator = Nominatim(user_agent="britto_app")
geopy.geocoders.options.default_timeout = 20

In [10]:
trilhas[12]

,lo_origem,la_origem,lo_destino,la_destino,velocidade_media,distancia,altitude_origem,altitude_destino,data,dia_da_semana,hora_origem,hora_destino,duracao
0,-38.386985,-12.938424,-38.462991,-12.998969,NaN,9.956643,14.100000,47.099998,2019-09-03,Terça,00:15:33.926000,09:48:49.200000,09:33:16
1,-38.462991,-12.998969,-38.460348,-12.999658,1.694010,0.299529,47.099998,12.000000,2019-09-03,Terça,09:48:49.200000,10:09:55,00:21:06
2,-38.460348,-12.999658,-38.463041,-12.997620,3.897991,0.347904,12.000000,0.000000,2019-09-03,Terça,10:09:55,10:10:56,00:01:01
3,-38.463041,-12.997620,-38.464523,-12.996605,4.122136,0.186820,0.000000,3.000000,2019-09-03,Terça,10:10:56,10:11:57,00:01:01
4,-38.464523,-12.996605,-38.464002,-12.994521,3.194965,0.190828,3.000000,3.000000,2019-09-03,Terça,10:11:57,10:12:57,00:01:00
5,-38.464002,-12.994521,-38.464248,-12.993147,2.691058,0.122979,3.000000,6.000000,2019-09-03,Terça,10:12:57,10:14:34,00:01:37
6,-38.464248,-12.993147,-38.464095,-12.991191,2.331272,0.171553,6.000000,16.000000,2019-09-03,Terça,10:14:34,10:15:35,00:01:01
7,-38.464095,-12.991191,-38.463090,-12.990961,1.395173,0.113283,16.000000,12.700001,2019-09-03,Terça,10:15:35,10:18:56.100000,00:03:21
8,-38.463090,-12.990961,-38.464020,-12.991196,1.299189,0.105179,12.700001,16.000000,2019-09-03,Terça,10:18:56.100000,10:20:55,00:01:59
9,-38.464020,-12.991196,-38.463170,-12.989856,3.104914,0.150208,16.000000,15.000000,2019-09-03,Terça,10:20:55,10:21:56,00:01:01


In [11]:
plt.plot(trilhas[12].lo_origem,trilhas[12].la_origem,color='burlywood')
mplleaflet.show()

In [12]:
# df_aux = trilhas[7]
cores = list(mcolors.CSS4_COLORS)
idx = 0
# plt.plot(trilhas[17].lo_origem,trilhas[17].la_origem,color=cores[0])
# plt.plot(trilhas[18].lo_origem,trilhas[18].la_origem,cores[1])
# plt.plot(trilhas[19].lo_origem,trilhas[19].la_origem,color=cores[2])
for trk in trilhas:
    print(str(idx) + '['+ str(len(trk))+']' + ' - ' + cores[idx] )
    plt.plot(trk.lo_origem,trk.la_origem,color=cores[idx])
    idx = idx+1
    
mplleaflet.show()

0[345] - aliceblue
1[126] - antiquewhite
2[52] - aqua
3[46] - aquamarine
4[78] - azure
5[89] - beige
6[53] - bisque
7[82] - black
8[76] - blanchedalmond
9[90] - blue
10[78] - blueviolet
11[69] - brown
12[59] - burlywood
13[85] - cadetblue
14[91] - chartreuse
15[78] - chocolate
16[77] - coral
17[31] - cornflowerblue
18[120] - cornsilk
19[100] - crimson
20[106] - cyan
21[70] - darkblue
22[73] - darkcyan
23[66] - darkgoldenrod
24[85] - darkgray
25[59] - darkgreen
26[142] - darkgrey
27[137] - darkkhaki
28[79] - darkmagenta
29[52] - darkolivegreen
30[69] - darkorange


In [ ]:
local = geolocator.reverse((-13.002414,-38.458652))
local

In [ ]:
local.altitude